# PoseBusters Checks

Run posebusters checks against a ligand and optional protein

In [ ]:
from posebusters import *
from rdkit import Chem
import pandas as pd

LIGAND_CHECKS = {
    "check_geometry": (check_geometry, [
        "bond_lengths_within_bounds", 
        "bond_angles_within_bounds", 
        "no_internal_clash",
    ]),
    "check_energy_ratio": (check_energy_ratio, ["energy_ratio_passes"]),
    "check_flatness": (check_flatness, ["flatness_passes"]),
    "check_chemistry": (check_chemistry, [
        "passes_rdkit_sanity_checks",
        # "inchi_convertible",
        "all_atoms_connected",
        "passes_valence_checks",
        "passes_kekulization",
    ]),
}

COMPLEX_CHECKS = {
    "check_volume_overlap":(check_volume_overlap, ["no_volume_clash"]),
}

def run_checks(ligand_file, protein_file=None):

    ligand = Chem.MolFromMolFile(ligand_file)
    protein = Chem.MolFromPDBFile(protein_file, removeHs=False)

    data = []

    overall_pass = True
    
    for check_name, (check, cols) in LIGAND_CHECKS.items():
        outcome = check(ligand)
        results = outcome["results"]
        d = {}
        d["check"] = check_name
        passes = True
        fails = set()
        for col in cols:
            if not (res := results[col]):
                passes = False
                overall_pass = False
                fails.add(col)
        d["pass"] = passes
        d["failed_checks"] = ", ".join(fails)
        data.append(d)

    if protein:
        for check_name, (check, cols) in COMPLEX_CHECKS.items():
            outcome = check(ligand, protein)
            results = outcome["results"]
            d = {}
            d["check"] = check_name
            passes = True
            fails = set()
            for col in cols:
                res = results[col]
                if not (res := results[col]):
                    passes = False
                    overall_pass = False
                    fails.add(col)
            d["pass"] = passes
            d["failed_checks"] = ", ".join(fails)
            data.append(d)

    df = pd.DataFrame(data)
    df = df.set_index("check")
    
    return overall_pass, df

In [ ]:
passed, df = run_checks(
    "A71EV2A/aligned_files/A0310a/A0310a_ligand.sdf",
    "A71EV2A/aligned_files/A0310a/A0310a_apo-desolv.pdb",
)
display(df)
passed